## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,CIty_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Barrow,71.2906,-156.7887,44.62,93,90,20.71,US,Fog
1,1,Dikson,73.5069,80.5464,48.00,97,100,10.00,RU,Clouds
2,2,Kikuyu,-1.2458,36.6633,57.69,65,90,8.34,KE,Clouds
3,3,Lavrentiya,65.5833,-171.0000,50.43,83,100,9.26,RU,Clouds
4,4,Ivanava,52.1451,25.5365,63.79,93,37,1.23,BY,Clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,CIty_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
7,7,Kirakira,-10.4544,161.9205,78.03,89,97,11.18,SB,Rain
12,12,Albany,42.6001,-73.9662,82.09,50,24,10.83,US,Clouds
13,13,Abu Samrah,35.3029,37.1841,78.91,56,0,11.36,SY,Clear
20,20,Broome,42.2506,-75.8330,80.02,59,9,1.01,US,Clear
28,28,Georgetown,5.4112,100.3354,78.89,91,20,3.44,MY,Clouds
32,32,Hithadhoo,-0.6000,73.0833,82.40,72,100,14.92,MV,Clouds
34,34,Shingu,33.7333,135.9833,80.91,90,100,16.24,JP,Clouds
39,39,Camopi,3.1655,-52.3158,80.10,82,98,2.98,GF,Clouds
42,42,Rawdon,46.0501,-73.7159,82.38,31,1,11.34,CA,Clear
54,54,Jamestown,42.0970,-79.2353,78.08,58,91,9.10,US,Clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

CIty_ID                129
City                   129
Lat                    129
Lng                    129
Max Temp               129
Humidity               129
Cloudiness             129
Wind Speed             129
Country                129
Current Description    129
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kirakira,SB,78.03,Rain,-10.4544,161.9205,
12,Albany,US,82.09,Clouds,42.6001,-73.9662,
13,Abu Samrah,SY,78.91,Clear,35.3029,37.1841,
20,Broome,US,80.02,Clear,42.2506,-75.8330,
28,Georgetown,MY,78.89,Clouds,5.4112,100.3354,
32,Hithadhoo,MV,82.40,Clouds,-0.6000,73.0833,
34,Shingu,JP,80.91,Clouds,33.7333,135.9833,
39,Camopi,GF,80.10,Clouds,3.1655,-52.3158,
42,Rawdon,CA,82.38,Clear,46.0501,-73.7159,
54,Jamestown,US,78.08,Clouds,42.0970,-79.2353,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kirakira,SB,78.03,Rain,-10.4544,161.9205,SanBiz Lodge
12,Albany,US,82.09,Clouds,42.6001,-73.9662,
13,Abu Samrah,SY,78.91,Clear,35.3029,37.1841,
20,Broome,US,80.02,Clear,42.2506,-75.8330,Chenango Valley State Park
28,Georgetown,MY,78.89,Clouds,5.4112,100.3354,Cititel Penang
...,...,...,...,...,...,...,...
526,Santa Cruz De La Palma,ES,79.14,Clouds,28.6835,-17.7642,H10 Taburiente Playa
533,Gizo,SB,81.23,Clouds,-8.1030,156.8419,Imagination Island
543,Taganak,PH,81.91,Clouds,6.0833,118.3000,Taganak
544,Bandarbeyla,SO,77.04,Clouds,9.4942,50.8122,JABIR HOTEL


In [22]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kirakira,SB,78.03,Rain,-10.4544,161.9205,SanBiz Lodge
20,Broome,US,80.02,Clear,42.2506,-75.8330,Chenango Valley State Park
28,Georgetown,MY,78.89,Clouds,5.4112,100.3354,Cititel Penang
32,Hithadhoo,MV,82.40,Clouds,-0.6000,73.0833,Scoop Guest House
34,Shingu,JP,80.91,Clouds,33.7333,135.9833,Shingu UI Hotel
...,...,...,...,...,...,...,...
526,Santa Cruz De La Palma,ES,79.14,Clouds,28.6835,-17.7642,H10 Taburiente Playa
533,Gizo,SB,81.23,Clouds,-8.1030,156.8419,Imagination Island
543,Taganak,PH,81.91,Clouds,6.0833,118.3000,Taganak
544,Bandarbeyla,SO,77.04,Clouds,9.4942,50.8122,JABIR HOTEL


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))